In [1]:
import os, getpass
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings

In [2]:
os.environ['PINECONE_API_KEY'] = getpass.getpass('Enter your Pinecone API key: ')
os.environ['OPENAI_API_KEY'] = getpass.getpass('Enter your OpenAI API key: ')

Enter your Pinecone API key:  ········
Enter your OpenAI API key:  ········


In [5]:
pinecone_api_key = os.environ.get('PINECONE_API_KEY')
openai_api_key = os.environ.get('OPENAI_API_KEY')

In [6]:
pc = Pinecone(api_key=pinecone_api_key)

In [7]:
index_name = "quinn-test"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="dotproduct",
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

In [8]:
# initializing a LangChain vector store
#model_name = 'text-embedding-ada-002'
embeddings = OpenAIEmbeddings()

In [9]:
vectorstore=PineconeVectorStore(index='quiddity-essesence1', embedding=embeddings)

In [10]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

# Path to an example text file
loader = TextLoader(r"C:\Users\Schalk\OneDrive - Columbia Business School\Coding\schalk-info.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

vectorstore_from_docs = PineconeVectorStore.from_documents(
    docs,
    index_name=index_name,
    embedding=embeddings,
    namespace = 'schalk-burger'
)   

In [11]:
# Example texts to add to the vector store
text = [
    "I like being honest",
    "I enjoy working hard and doing math",
    "I like golf"
]

# Create a vector store from texts
vectorstore_from_texts = PineconeVectorStore.from_texts(
    text,
    index_name=index_name,
    embedding=embeddings,
    namespace = 'schalk-burger'
)

In [12]:
query = "How old am I?"
vectorstore.similarity_search(query, k=2, namespace='schalk-burger')

AttributeError: 'str' object has no attribute 'query'

In [23]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

In [24]:
llm = ChatOpenAI(
    openai_api_key=openai_api_key,
    model="gpt-3.5-turbo-0125",
    temperature=0.0
)